<a href="https://colab.research.google.com/github/chaitraDev/GenAI-for-marketing/blob/main/Cognizant_RFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import datetime
# Set random seed for reproducibility
np.random.seed(42)
isGenerating = False #flag to choose between imported and generated data

# Data Generation

In [ ]:
#@title Customers data
def generate_customers(fake,num_customers=100):
  customer_data = {
      "CustomerID": [f"CUST{i:03d}" for i in range(1, num_customers + 1)],
      "CustGender": np.random.choice(["Male", "Female"], num_customers),
      "Age": np.random.randint(18, 70, num_customers),
      "Income": np.random.randint(30000, 150000, num_customers),
      "Loan": np.random.choice(["Yes", "No"], num_customers),
      "Num_Transactions": np.random.randint(1, 20, num_customers),
      # "balance": np.random.randint(1000, 50000, num_customers),
      "Cibil_Score": np.random.randint(300, 900, num_customers),
      # "cust_joining_date": [fake.date_this_decade() for _ in range(num_customers)],
      "Credit_Card": np.random.choice(["Yes", "No"], num_customers),
      "Debit_Card": np.random.choice(["Yes", "No"], num_customers),
      "Product":[]
  }
  customers_df = pd.DataFrame(customer_data)
  customers_df.sample(frac=1,ignore_index=True)
  return customers_df

In [ ]:
#@title Products data
def generate_products(fake,num_products=9):
  products = ["Education Loan", "Vehicle Loan", "Home Loan", "Current Account", "Savings Account",
              "Credit Card", "Debit Card", "Recurring Deposit", "Fixed Deposit"]
  product_data = {
      "p_id": [f"PROD{i:03d}" for i in range(1, num_products + 1)],
      "p_name": [fake.bs() for _ in range(num_products)],
      "p_type": np.random.choice(products, num_products),
      "p_el_age": np.random.randint(18, 70, num_products),
      "p_el_income": np.random.randint(30000, 150000, num_products),
      "p_el_cibil": np.random.randint(300, 900, num_products)
  }
  products_df = pd.DataFrame(product_data)
  products_df.sample(frac=1,ignore_index=True)
  return products_df

In [ ]:
#@title Transactions data
def generate_transactions(fake,customers_df,num_transactions=1000):
  transaction_types = ["Loan Amount Disbursement", "Loan Installment", "Credit", "Debit", "FD", "RD"]
  transaction_data = {
      "TransactionID": [],
      "CustomerID": [],
      "TransactionDate": [],
      "CustAccountBalance": [],
      "TransactionAmount (INR)": [],
      "t_type": [],
      "TransactionTime": [],
      "payment_method": []
  }

  trans_count = 1
  num_customers = customers_df.shape[0]
  cust_ids = customers_df["CustomerID"]
  payment_methods = ["Credit Card", "Debit Card","UPI", "Net Banking", "Cash"]
  for index in range(num_customers):
    cust_id = cust_ids[index]
    customer = customers_df[customers_df["CustomerID"] == cust_id].iloc[0]
    cust_trans = customer["Num_Transactions"]

    for _ in range(1, cust_trans + 1):
      t_id = f"TRANS{trans_count:04d}"
      trans_count += 1
      cust_id = cust_ids[index]

      t_date = fake.date_this_decade()
      cust_acc_balance = np.random.randint(1000, 50000)
      t_amt = round(np.random.uniform(10.0, 5000.0),2)
      t_time = fake.time()

    # ----- check for loan add only those transactions ----- #
      if customer["Loan"] == "Yes":
        t_type = np.random.choice(transaction_types)
        # add_loan()
      else:
        t_type = np.random.choice(transaction_types[2:])

      # ----- check for cards to append in payment method ----- #
      if customer["Credit_Card"] == "Yes" and customer["Debit_Card"] == "Yes" :
          payment_method = np.random.choice(payment_methods)
      elif customer["Credit_Card"] == "Yes":#no debit card
          payment_method = np.random.choice(["Credit Card", "UPI", "Net Banking", "Cash"])
      elif customer["Debit_Card"] == "Yes":#no credit card
          payment_method = np.random.choice(["Debit Card", "UPI", "Net Banking", "Cash"])
      else: #no debit or credit card
          payment_method = np.random.choice(["UPI", "Net Banking", "Cash"])

      transaction_data["TransactionID"].append(t_id)
      transaction_data["CustomerID"].append(cust_id)
      transaction_data["TransactionDate"].append(t_date)
      transaction_data["CustAccountBalance"].append(cust_acc_balance)
      transaction_data["TransactionAmount (INR)"].append(t_amt)
      transaction_data["t_type"].append(t_type)
      transaction_data["TransactionTime"].append(t_time)
      transaction_data["payment_method"].append(payment_method)


  transactions_df = pd.DataFrame(transaction_data)
  #converting date column to datetime.date object
  transactions_df.sample(frac=1,ignore_index=True)
  return transactions_df

In [ ]:
def fake_data():
  !pip install faker
  from faker import Faker
  # Initialize Faker
  fake = Faker()

  customers = generate_customers(fake)
  products = generate_products(fake)
  transactions = generate_transactions(fake,customers)

  transactions["TransactionDate"] = pd.to_datetime(transactions.TransactionDate).dt.date

  customers.to_csv("customers.csv", index=False)
  products.to_csv("products.csv", index=False)
  transactions.to_csv("transactions.csv", index=False)

  return customers,transactions,products

# Data Import

In [ ]:
def import_data():
  customers = pd.read_csv("/content/drive/MyDrive/Cognizant/customers.csv")
  transactions = pd.read_csv("/content/drive/MyDrive/Cognizant/transactions.csv")
  transactions["TransactionDate"] = pd.to_datetime(transactions.TransactionDate).dt.date
  return customers, transactions

# EDA

In [ ]:
if not isGenerating:
  # to take the imported data
  customers,transactions = import_data()
else:
  # to take the generated data
  customers,transactions,products = fake_data()

<ipython-input-7-25b34d319d63>:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  transactions["TransactionDate"] = pd.to_datetime(transactions.TransactionDate).dt.date


In [ ]:
customers.head()

,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance
0,C5841053,10/1/94,F,JAMSHEDPUR,17819.05
1,C2142763,4/4/57,M,JHAJJAR,2270.69
2,C4417068,26/11/96,F,MUMBAI,17874.44
3,C5342380,14/9/73,F,MUMBAI,866503.21
4,C9031234,24/3/88,F,NAVI MUMBAI,6714.43


In [ ]:
transactions.head()

,TransactionID,CustomerID,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,2016-02-08,143207,25.0
1,T2,C2142763,2016-02-08,141858,27999.0
2,T3,C4417068,2016-02-08,142712,459.0
3,T4,C5342380,2016-02-08,142714,2060.0
4,T5,C9031234,2016-02-08,181156,1762.5


In [ ]:
# products.head()

In [ ]:
customers.isnull().sum()

,0
CustomerID,0
CustomerDOB,1747
CustGender,540
CustLocation,87
CustAccountBalance,1195


In [ ]:
transactions["CustomerID"].value_counts()

,count
CustomerID,
C7537344,6
C7529869,5
C6228352,5
C8019227,5
C6331068,5
...,...
C5634561,1
C8318292,1
C3942563,1


# Quantifying Customer Relation

## Preparing the RFM columns

In [ ]:
transactions.dtypes

,0
TransactionID,object
CustomerID,object
TransactionDate,object
TransactionTime,int64
TransactionAmount (INR),float64


In [ ]:
recents = transactions.groupby('CustomerID')["TransactionDate"].max().reset_index()
# left join customers table to transactions table
customers = customers.merge(recents,on="CustomerID",how="left")
customers.rename(columns = {"TransactionDate":"most_recent"}, inplace=True)
customers.columns

Index(['CustomerID', 'CustomerDOB', 'CustGender', 'CustLocation',
       'CustAccountBalance', 'most_recent'],
      dtype='object')

In [ ]:
today = datetime.datetime.today().date()
customers["Gap"] = transactions["TransactionDate"].apply(lambda date: (today-date).days)
customers["Num_Transactions"] = transactions.groupby("CustomerID")["TransactionID"].count().values
customers["Sum_Transactions"] = transactions.groupby("CustomerID")["TransactionAmount (INR)"].sum().values

In [ ]:
customers.columns

Index(['CustomerID', 'CustomerDOB', 'CustGender', 'CustLocation',
       'CustAccountBalance', 'most_recent', 'Gap', 'Num_Transactions',
       'Sum_Transactions'],
      dtype='object')

## Scoring RFM columns between 1-5, based on quartiles

In [ ]:
customers["Recency"] = pd.qcut(customers["Gap"],q=5,labels=[5,4,3,2,1]).astype(int)
if not isGenerating:
  customers["Frequency"] = customers["Num_Transactions"]
else:
  customers["Frequency"] = pd.qcut(customers["Num_Transactions"],q=5,labels=[1,2,3,4,5]).astype(int)
customers["Monetary_Value"] = pd.qcut(customers["Sum_Transactions"],q=5,labels=[1,2,3,4,5]).astype(int)

In [ ]:
customers["Recency"],customers["Frequency"],customers["Monetary_Value"]

(0         1
 1         1
 2         1
 3         1
 4         1
          ..
 539671    4
 539672    4
 539673    4
 539674    4
 539675    4
 Name: Recency, Length: 539676, dtype: int64,
 0         1
 1         2
 2         1
 3         1
 4         1
          ..
 539671    1
 539672    1
 539673    1
 539674    1
 539675    1
 Name: Frequency, Length: 539676, dtype: int64,
 0         5
 1         4
 2         1
 3         5
 4         3
          ..
 539671    3
 539672    1
 539673    1
 539674    2
 539675    4
 Name: Monetary_Value, Length: 539676, dtype: int64)

In [ ]:
customers["RFM_Score"] = customers["Recency"]*100 + customers["Frequency"]*10 + customers["Monetary_Value"]

In [ ]:
customers.head()

,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,most_recent,Gap,Num_Transactions,Sum_Transactions,Recency,Frequency,Monetary_Value,RFM_Score
0,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2016-02-08,3111,1,4750.0,1,1,5,115
1,C2142763,4/4/57,M,JHAJJAR,2270.69,2016-02-08,3111,2,1455.0,1,2,4,124
2,C4417068,26/11/96,F,MUMBAI,17874.44,2016-02-08,3111,1,30.0,1,1,1,111
3,C5342380,14/9/73,F,MUMBAI,866503.21,2016-08-13,3111,1,5000.0,1,1,5,115
4,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2016-02-08,3111,1,557.0,1,1,3,113


# RFM Scoring

In [ ]:
# the scores range from 111(lowest) to 666(highest), so we can divide in 3 parts
# as 111-259(low), 260-408(med), 409-666(high)

customers["Relationship_Value"] = pd.cut(customers["RFM_Score"], bins=[110, 259, 408, 566], labels=["low", "med", "high"])
customers["Relationship_Value"].head()

,Relationship_Value
0,low
1,low
2,low
3,low
4,low


In [ ]:
customers.head()

,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,most_recent,Gap,Num_Transactions,Sum_Transactions,Recency,Frequency,Monetary_Value,RFM_Score,Relationship_Value
0,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2016-02-08,3111,1,4750.0,1,1,5,115,low
1,C2142763,4/4/57,M,JHAJJAR,2270.69,2016-02-08,3111,2,1455.0,1,2,4,124,low
2,C4417068,26/11/96,F,MUMBAI,17874.44,2016-02-08,3111,1,30.0,1,1,1,111,low
3,C5342380,14/9/73,F,MUMBAI,866503.21,2016-08-13,3111,1,5000.0,1,1,5,115,low
4,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2016-02-08,3111,1,557.0,1,1,3,113,low


In [ ]:
customers["Relationship_Value"].value_counts()

,count
Relationship_Value,
high,221664
low,211232
med,106780


In [ ]:
# Getting the segmentwise customer ID
lows = customers[customers.Relationship_Value=="low"]["CustomerID"]
meds = customers[customers.Relationship_Value=="med"]["CustomerID"]
high = customers[customers.Relationship_Value=="high"]["CustomerID"]
cust_values = {"low":lows,"medium":meds,"high":high}

# Storing

In [ ]:
if not isGenerating:
  customers.to_csv("cust_segmented.csv",index=False)
else:
  extra_cols = ["Loan","Credit_Card","Debit_Card"]
  customers.drop(columns=extra_cols,inplace=True)
  customers.to_csv("fake_cust.csv",index=False)

In [ ]:
customers.isnull().sum()

,0
CustomerID,0
CustomerDOB,1747
CustGender,540
CustLocation,87
CustAccountBalance,1195
most_recent,0
Gap,0
Num_Transactions,0
Sum_Transactions,0
Recency,0


# Showing process

## Original Data

In [ ]:
transactions[["CustomerID","TransactionDate","TransactionAmount (INR)"]].head()

,CustomerID,TransactionDate,TransactionAmount (INR)
0,C5841053,2016-02-08,25.0
1,C2142763,2016-02-08,27999.0
2,C4417068,2016-02-08,459.0
3,C5342380,2016-02-08,2060.0
4,C9031234,2016-02-08,1762.5


## Finding Most Recent Transaction, transaction count, total amount for each customer, from transactions data

In [ ]:
recents = transactions.groupby('CustomerID')["TransactionDate"].max().reset_index()
# left join customers table to transactions table
customers = customers.merge(recents,on="CustomerID",how="left")
customers.rename(columns = {"TransactionDate":"most_recent"}, inplace=True)

In [ ]:
today = datetime.datetime.today().date()
customers["Gap"] = transactions["TransactionDate"].apply(lambda date: (today-date).days)
customers["Num_Transactions"] = transactions.groupby("CustomerID")["TransactionID"].count().values
customers["Sum_Transactions"] = transactions.groupby("CustomerID")["TransactionAmount (INR)"].sum().values

In [ ]:
customers[["CustomerID","most_recent","Num_Transactions","Sum_Transactions"]].head()

,CustomerID,most_recent,most_recent,Num_Transactions,Sum_Transactions
0,C5841053,2016-02-08,2016-02-08,1,4750.0
1,C2142763,2016-02-08,2016-02-08,2,1455.0
2,C4417068,2016-02-08,2016-02-08,1,30.0
3,C5342380,2016-08-13,2016-08-13,1,5000.0
4,C9031234,2016-02-08,2016-02-08,1,557.0


## Getting Recency, Frequency, Monetary_Value from those columns, by cutting them in intervals of 1-5, (1-6 for frequency)

In [ ]:
customers["Recency"] = pd.qcut(customers["Gap"],q=5,labels=[5,4,3,2,1]).astype(int)
customers["Frequency"] = customers["Num_Transactions"] # this value is already in range of 1-6
customers["Monetary_Value"] = pd.qcut(customers["Sum_Transactions"],q=5,labels=[1,2,3,4,5]).astype(int)

In [ ]:
customers[["CustomerID","Recency","Frequency","Monetary_Value"]].head()

,CustomerID,Recency,Frequency,Monetary_Value
0,C5841053,1,1,5
1,C2142763,1,2,4
2,C4417068,1,1,1
3,C5342380,1,1,5
4,C9031234,1,1,3


## Getting RFM score from individual columns and assigning relationship value based on this score

In [ ]:
customers["RFM_Score"] = customers["Recency"]*100 + customers["Frequency"]*10 + customers["Monetary_Value"]

In [ ]:
# the scores range from 111(lowest) to 666(highest), so we can divide in 3 parts
# as 111-259(low), 260-408(med), 409-666(high)

customers["Relationship_Value"] = pd.cut(customers["RFM_Score"], bins=[110, 259, 408, 666], labels=["low", "med", "high"])

In [ ]:
customers[["CustomerID","RFM_Score","Relationship_Value"]].head()

,CustomerID,RFM_Score,Relationship_Value
0,C5841053,115,low
1,C2142763,124,low
2,C4417068,111,low
3,C5342380,115,low
4,C9031234,113,low


# Visualizing

In [ ]:
customers,transactions = import_data()

<ipython-input-3-25b34d319d63>:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  transactions["TransactionDate"] = pd.to_datetime(transactions.TransactionDate).dt.date


In [ ]:
import seaborn as sns
customers.Product.value_counts()

AttributeError: 'DataFrame' object has no attribute 'Product'